In [1]:

import pandas as pd


df_clientes = pd.read_csv("datasets/olist_customers_dataset.csv")
df_items = pd.read_csv("datasets/olist_order_items_dataset.csv")
df_nomes = pd.read_csv("datasets/product_category_name_translation.csv")
df_pedidos = pd.read_csv("datasets/olist_orders_dataset.csv")
df_vendedores = pd.read_csv("datasets/olist_sellers_dataset.csv")
df_pagamentos = pd.read_csv("datasets/olist_order_payments_dataset.csv")
df_produtos = pd.read_csv("datasets/olist_products_dataset.csv")

In [2]:
df_pedidos = df_pedidos.dropna(subset=['order_delivered_carrier_date', 'order_delivered_customer_date'])


In [3]:
df_items['shipping_limit_date'] = pd.to_datetime(df_items['shipping_limit_date'])
df_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [4]:
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_approved_at'] = pd.to_datetime(df_pedidos['order_approved_at'])
df_pedidos['order_delivered_carrier_date'] = pd.to_datetime(df_pedidos['order_delivered_carrier_date'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])
df_pedidos['order_estimated_delivery_date'] = pd.to_datetime(df_pedidos['order_estimated_delivery_date'])


In [5]:
df_pedidos['is_delivered_late'] = df_pedidos['order_delivered_customer_date'] > df_pedidos['order_estimated_delivery_date']

In [6]:
df_vendas = df_items.groupby("seller_id").agg(
    sales=("order_id", "count"),
    total_sold=("price", "sum"),
)


df_vendas["medium_ticket"] = df_vendas["total_sold"] / df_vendas["sales"]

df_vendas.reset_index(inplace=True)

df_vendedores = df_vendedores.merge(df_vendas, on="seller_id", how="left")

df_vendedores.fillna({"sales": 0, "total_sold": 0, "medium_ticket": 0}, inplace=True)

print(df_vendedores.head())


                          seller_id  seller_zip_code_prefix  \
0  3442f8959a84dea7ee197c632cb2df15                   13023   
1  d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
2  ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
3  c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
4  51a04a8a6bdcb23deccc82b0b80742cf                   12914   

         seller_city seller_state  sales  total_sold  medium_ticket  
0           campinas           SP      3      218.70      72.900000  
1         mogi guacu           SP     41    11703.07     285.440732  
2     rio de janeiro           RJ      1      158.00     158.000000  
3          sao paulo           SP      1       79.99      79.990000  
4  braganca paulista           SP      1      167.99     167.990000  


In [7]:
df_produto_vendas = df_items.groupby("product_id").agg(
    total_sales=("order_id", "count"),
    total_revenue=("price", "sum")
)

df_produto_vendas.reset_index(inplace=True)


df_produtos = df_produtos.merge(df_produto_vendas, on="product_id", how="left")

df_produtos.fillna({"total_sales": 0, "total_revenue": 0}, inplace=True)

print(df_produtos.head())

                         product_id  product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes   
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer   
3  cef67bcfe19066a932b7673e239eb23d                  bebes   
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   
2                 46.0                       250.0                 1.0   
3                 27.0                       261.0                 1.0   
4                 37.0                       402.0                 4.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  \
0             225.0               16.0               10.0              14.0   
1            1000.0     

In [11]:
df_order_totals = df_items.groupby('order_id')['price'].sum().reset_index()
df_order_totals.rename(columns={'price': 'total_order_value'}, inplace=True)

df_pedidos = df_pedidos.merge(df_order_totals, on='order_id', how='left')


df_customer_stats = df_pedidos.groupby('customer_id').agg(
    total_spent=('total_order_value', 'sum'),
    average_order_value=('total_order_value', 'mean'),
    total_orders=('order_id', 'count')
).reset_index()

df_clientes = df_clientes.merge(df_customer_stats, on='customer_id', how='left')

df_clientes.fillna({'total_spent': 0, 'average_order_value': 0, 'total_orders': 0}, inplace=True)


In [ ]:
receita_total = df_items['price'].sum()

vendas_por_estado = df_pedidos.merge(df_clientes, on='customer_id').groupby('customer_state')['order_id'].count()

vendas_por_categoria = df_items.merge(df_produtos, on='product_id').groupby('product_category_name')['order_id'].count()

df_resumo = pd.DataFrame({
    'Receita Total': [receita_total],
    'Vendas por Estado': [vendas_por_estado.to_dict()],
    'Vendas por Categoria': [vendas_por_categoria.to_dict()]
})

print(df_resumo)


   Receita Total                                  Vendas por Estado  \
0     13591643.7  {'AC': 80, 'AL': 397, 'AM': 145, 'AP': 67, 'BA...   

                                Vendas por Categoria  
0  {'agro_industria_e_comercio': 212, 'alimentos'...  


In [ ]:
dataset_ref = client.dataset("ecommerce_analise")  # Nome do novo dataset
try:
    client.get_dataset(dataset_ref)  # Verifica se já existe
    print("Dataset já existe.")
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Escolha a região desejada
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_ref} criado com sucesso.")


Dataset corded-shadow-453116-f7.ecommerce_analise criado com sucesso.


In [ ]:

tabela_resumo_ref = client.dataset("ecommerce_analise").table("resumo_vendas")

job = client.load_table_from_dataframe(df_resumo, tabela_resumo_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

job.result()
print("Tabela resumo_vendas carregada com sucesso!")


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Vendas por Estado, Vendas por Categoria.
  warnings.warn(


Tabela resumo_vendas carregada com sucesso!


In [ ]:
# Carregar vendedores transformados
tabela_vendedores_ref = client.dataset("ecommerce_analise").table("vendedores")
job_vendedores = client.load_table_from_dataframe(df_vendedores, tabela_vendedores_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job_vendedores.result()

# Carregar clientes transformados
tabela_clientes_ref = client.dataset("ecommerce_analise").table("clientes")
job_clientes = client.load_table_from_dataframe(df_clientes, tabela_clientes_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job_clientes.result()

print("Todas as tabelas transformadas foram carregadas no BigQuery com sucesso!")


Todas as tabelas transformadas foram carregadas no BigQuery com sucesso!


In [13]:
df_resumo

,Receita Total,Vendas por Estado,Vendas por Categoria
0,13591643.7,"{'AC': 80, 'AL': 397, 'AM': 145, 'AP': 67, 'BA...","{'agro_industria_e_comercio': 212, 'alimentos'..."


In [16]:
import pandas as pd
import streamlit as st
import plotly.express as px

# Calcular Receita Total
receita_total = df_items["price"].sum()

# Número de Vendas por Estado
df_vendas_estado = df_pedidos.merge(df_clientes, on="customer_id").groupby("customer_state")["order_id"].count().reset_index()
df_vendas_estado.columns = ["estado", "num_vendas"]

# Número de Vendas por Categoria de Produto
df_vendas_categoria = df_items.merge(df_produtos, on="product_id").groupby("product_category_name")["order_id"].count().reset_index()
df_vendas_categoria.columns = ["categoria_produto", "total_vendas"]

# Criar DataFrame Resumo
df_resumo = pd.DataFrame({"Receita Total": [receita_total]})

# Criar o dashboard
st.title("Dashboard de Vendas - E-commerce")

# Receita Total
st.metric(label="Receita Total", value=f"R$ {receita_total:,.2f}")

# Gráfico de Número de Vendas por Estado
fig_estado = px.bar(df_vendas_estado, x="estado", y="num_vendas", title="Vendas por Estado")
st.plotly_chart(fig_estado)

# Gráfico de Vendas por Categoria de Produto
fig_categoria = px.pie(df_vendas_categoria, names="categoria_produto", values="total_vendas", title="Vendas por Categoria")
st.plotly_chart(fig_categoria)



2025-03-09 00:58:05.649 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.651 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-09 00:58:05.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()